In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [4]:
alpaca_prompt = """Your role is to act as an expert in HLS C++ code development. You must thoroughly explore each question through a systematic and deliberate thinking process—engaging in cycles of analysis, summarization, exploration, reassessment, reflection, backtracing, and iteration—to develop well-considered solutions. Based on the provided instructions, you are expected to generate precise, optimized, and accurate HLS C++ code that meets the requirements. Additionally, you will be provided with the Abstract Syntax Tree (AST) of the HLS code to assist in generating a more optimized and correct implementation. You should leverage the AST to ensure accuracy, efficiency, and adherence to best practices in hardware synthesis.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
  texts = []
  ast_prompt = "\n\nHere is the AST (Abstract Syntax Tree) of the HLS code to assist in generating an optimized and correct implementation:\n\n" 
  for instruction, hls_ast, response in zip(examples['hls_instruction'], examples['hls_ast'], examples['hls_code']):
      input_text = ""
      instruction += ast_prompt
      instruction += hls_ast
      instruction += "\n"
      
      formatted_text = alpaca_prompt.format(instruction, input_text, response) + EOS_TOKEN
      texts.append(formatted_text)


  return { "text" : texts}


from datasets import load_dataset
dataset = load_dataset("mashnoor/hls-vitis-ast-unverified", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
dataset[0]['text']

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, 
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.push_to_hub("mashnoor/hls-vitis-ast-unverified-qwen-coder-7B", commit_message="First upload") 
tokenizer.push_to_hub("mashnoor/hls-vitis-ast-unverified-qwen-coder-7B", commit_message="Upload version v1.0") 

# model.save_pretrained("hls-vitis-unverified-2")  
# tokenizer.save_pretrained("hls-vitis-unverified-2")

In [11]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        '''
        Your role is to act as an expert in HLS C++ code development. You must thoroughly explore each question through a systematic and deliberate thinking process—engaging in cycles of analysis, summarization, exploration, reassessment, reflection, backtracing, and iteration—to develop well-considered solutions. Based on the provided instructions, you are expected to generate precise, optimized, and accurate HLS C++ code that meets the requirements.

        ''',
        '''
        Make a two bit adder
        ''', 
        "", 
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 30000)